In [1]:
#add the path of my package `DPMacro`
import sys
sys.path.append('/home/zhenfeng/ProteinFeature_1/DPMacro')
import AFill


In [2]:
a=AFill.structure_aligner()

In [4]:
a._set_structure('multimer_af2_afm_ft_0506_c16_sd0.5_s512_4k.pt_9871_r11_e1.pdb')
a.load_reference('1h2r.pdb')



In [5]:
#check a.sequences and a.reference_sequences for existing chain 
print(a.sequences)
print(a.reference_sequences)

{'A': 'MASNGHSAGQNAIDELPDISPVDGIRRRLDDPQVAEALNSLLDHADLLAVLVKGLDGFVRRGDDIANNLTSAIGELKALNAADTPIPALAALKDVDLAGLANSLATLSGGLVKATPALNAVLDSLTDQRGAEVLSALGDALVAARTSAPPAPRGVRGMWKTLRAAAKDPDVGRGVSYLIEVARVFGSKV', 'B': 'MSAWSHPQFEKGGGSGGGSGGSAWSHPQFEKSGGGGGENLYFQGSGGASVLWFQGGACSGNTMSFLNADEPNVVDLIVDFGLDLLWHPSLGLELGNNAQKVFWDCAKGERPLDIFVFEGTVIEAPNGTGQMDMFAGRPMKDWVTDLAGAAQIVVAIGDCACFGGIPAMEPNPSGSTGLQFHKREKGGFLGPDFRSKMGLPVINVPGCPAHPDWITQILVALATGRAGDITLDDLHRPETFFKTFTQTGCTRVQFFEYKQSTLSFGEGTRTGCLFYEFGCRGPMTHSPCNRILWNRQSSKTRAGMPCLGCTEPEFPHFDLAPGTVFKTQKVSGMIPKEVPEGTDHLTYMGLAAAARIAAPQWSKEDMFVV', 'C': 'MSAWSHPQFEKGGGSGGGSGGSAWSHPQFEKSGGGGGENLYFQGSGGASVLWFQGGACSGNTMSFLNADEPNVVDLIVDFGLDLLWHPSLGLELGNNAQKVFWDCAKGERPLDIFVFEGTVIEAPNGTGQMDMFAGRPMKDWVTDLAGAAQIVVAIGDCACFGGIPAMEPNPSGSTGLQFHKREKGGFLGPDFRSKMGLPVINVPGCPAHPDWITQILVALATGRAGDITLDDLHRPETFFKTFTQTGCTRVQFFEYKQSTLSFGEGTRTGCLFYEFGCRGPMTHSPCNRILWNRQSSKTRAGMPCLGCTEPEFPHFDLAPGTVFKTQKVSGMIPKEVPEGTDHLTYMGLAAAARIAAPQWSKEDMFVV', 'D': 'MTELDLFVSPLGRVEGDLDVRVTINDGVVTSAWTEAAMF

In [6]:
#check a.ligands for available ligands
a.ligands

[<Residue SF4 het=H_SF4 resseq=1001 icode= >,
 <Residue SF4 het=H_SF4 resseq=1002 icode= >,
 <Residue F3S het=H_F3S resseq=1003 icode= >,
 <Residue MG het=H_MG resseq=1005 icode= >,
 <Residue NFE het=H_NFE resseq=1004 icode= >]

In [7]:
#choose a ligand for downsteam job
a.set_ligand(a.ligands[0])



In [8]:
#use a.neighbor to see residue within 6 a of the chosen ligand.
a.neighbor

{<Residue ARG het=  resseq=59 icode= >,
 <Residue ARG het=  resseq=79 icode= >,
 <Residue GLY het=  resseq=82 icode= >,
 <Residue HIS het=  resseq=235 icode= >,
 <Residue ALA het=  resseq=15 icode= >,
 <Residue GLU het=  resseq=16 icode= >,
 <Residue CYS het=  resseq=17 icode= >,
 <Residue THR het=  resseq=18 icode= >,
 <Residue GLY het=  resseq=19 icode= >,
 <Residue CYS het=  resseq=20 icode= >,
 <Residue GLU het=  resseq=75 icode= >,
 <Residue GLY het=  resseq=76 icode= >,
 <Residue TYR het=  resseq=111 icode= >,
 <Residue GLY het=  resseq=112 icode= >,
 <Residue THR het=  resseq=113 icode= >,
 <Residue CYS het=  resseq=114 icode= >,
 <Residue ALA het=  resseq=115 icode= >,
 <Residue GLY het=  resseq=119 icode= >,
 <Residue VAL het=  resseq=120 icode= >,
 <Residue GLN het=  resseq=121 icode= >,
 <Residue GLY het=  resseq=149 icode= >,
 <Residue CYS het=  resseq=150 icode= >,
 <Residue PRO het=  resseq=151 icode= >,
 <Residue PRO het=  resseq=152 icode= >,
 <Residue TRP het=  resseq=

In [11]:
# or a more clear version will be like:
# from util import  _integrated_residue_iterator
# for i in _integrated_residue_iterator(a.neighbor):
#     print(i.get_full_id()[1:] )

In [9]:
#select neighbors in pymol
AFill.reslist_to_guicode(a.neighbor,'pymol')

'(chain S and resi 113 ) or (chain S and resi 17 ) or (chain S and resi 258 ) or (chain S and resi 150 ) or (chain S and resi 119 ) or (chain L and resi 59 ) or (chain S and resi 20 ) or (chain L and resi 235 ) or (chain S and resi 19 ) or (chain S and resi 115 ) or (chain S and resi 112 ) or (chain S and resi 149 ) or (chain S and resi 75 ) or (chain S and resi 152 ) or (chain S and resi 121 ) or (chain S and resi 16 ) or (chain S and resi 15 ) or (chain S and resi 111 ) or (chain L and resi 82 ) or (chain L and resi 79 ) or (chain S and resi 114 ) or (chain S and resi 151 ) or (chain S and resi 120 ) or (chain S and resi 18 ) or (chain S and resi 76 )'

In [10]:
# the neighbor contains chain S & L,so we will go to map thses 2 chain in our model to target.
# so far, which chain in our model is mapped to concerned chain rely on visualization check.
# but I'm dealing with it,try to make it more smart.
a._align('B','S')
a.map_neighbor()
a._align('D','L')
a.map_neighbor()


In [34]:
a.alignment.sc

{'_records': [SeqRecord(seq=Seq('-MTELDLFVSPLGRVEGDLDVRVTINDGVVTSAWTEAAMFRGFEIILRGKDPQA...---'), id='multimer_af2_afm_ft_0506_c16_s', name='<unknown name>', description='multimer_af2_afm_ft_0506_c16_s', dbxrefs=[]),
  SeqRecord(seq=Seq('SSYSGPIVVDPVTRIEGHLRIEVEVENGKVKNAYSSSTLFRGLEIILKGRDPRD...XXX'), id='1h2r.pdb_L', name='<unknown name>', description='1h2r.pdb_L', dbxrefs=[])],
 '_per_col_annotations': {'clustal_consensus': '   .  :.*.*: *:**.* :.* :::* *..*::.:::***:****:*:**: .  .  * **:*  :*   *: .:*.*  .*:* *** ***:  ..: *:.   :** *.*:*:.: . * .    :*.:  :.    .  .. : :.:      :  **    :     *  .*  *.  *. **  .: :   *: :   . .    :* :*   *:. *   ::   .  . *     :.        : : *...: ::     . * :  . . : :.    :.*:*:              **: :: *.* **:*** .. **::*:* *  . * : :.:*** *:***        *:******: : * :  ..:.:   *.::*.:   :**..:.  *.*                  :::: *::   :..   * *   :.:*.* .:*.**.**.** ***.** *:*:*.*::*.:***  .: :.*:* :*:*:*:.**: ***: :..:***.*:.* **                             

In [11]:
#check a.align or a.alignment_frame for full residue map
#right now,a.align / a.alignment_frame will be overwrite when run a._align().I'm repairing it

#check a.mapped_neighbor for mapped pocket.
a.mapped_neighbor


[<Residue GLY het=  resseq=56 icode= >,
 <Residue ALA het=  resseq=57 icode= >,
 <Residue CYS het=  resseq=58 icode= >,
 <Residue SER het=  resseq=59 icode= >,
 <Residue GLY het=  resseq=60 icode= >,
 <Residue ASN het=  resseq=61 icode= >,
 <Residue GLU het=  resseq=118 icode= >,
 <Residue GLY het=  resseq=119 icode= >,
 <Residue ILE het=  resseq=156 icode= >,
 <Residue GLY het=  resseq=157 icode= >,
 <Residue ASP het=  resseq=158 icode= >,
 <Residue CYS het=  resseq=159 icode= >,
 <Residue ALA het=  resseq=160 icode= >,
 <Residue GLY het=  resseq=164 icode= >,
 <Residue ILE het=  resseq=165 icode= >,
 <Residue PRO het=  resseq=166 icode= >,
 <Residue GLY het=  resseq=206 icode= >,
 <Residue CYS het=  resseq=207 icode= >,
 <Residue PRO het=  resseq=208 icode= >,
 <Residue ALA het=  resseq=209 icode= >,
 <Residue XXX het=place_holder resseq=0 icode= >,
 <Residue ARG het=  resseq=40 icode= >,
 <Residue ARG het=  resseq=60 icode= >,
 <Residue GLY het=  resseq=63 icode= >,
 <Residue LYS he

In [12]:

AFill.reslist_to_guicode(a.mapped_neighbor,'pymol')


'(chain B and resi 56 ) or (chain B and resi 57 ) or (chain B and resi 58 ) or (chain B and resi 59 ) or (chain B and resi 60 ) or (chain B and resi 61 ) or (chain B and resi 118 ) or (chain B and resi 119 ) or (chain B and resi 156 ) or (chain B and resi 157 ) or (chain B and resi 158 ) or (chain B and resi 159 ) or (chain B and resi 160 ) or (chain B and resi 164 ) or (chain B and resi 165 ) or (chain B and resi 166 ) or (chain B and resi 206 ) or (chain B and resi 207 ) or (chain B and resi 208 ) or (chain B and resi 209 ) or (chain D and resi 40 ) or (chain D and resi 60 ) or (chain D and resi 63 ) or (chain D and resi 215 )'

In [ ]:
读进来，sele，align，写出（一部分），拼结构
from pymol import cmd
cmd.load()
cmd.select(name='1a')
cmd.align(mobile=,target=)
cmd.save(,selection='1a')
cmd.delete('all')


In [33]:
from util import read_in,_integrated_residue_iterator
s=read_in('1h2r.pdb')
s.atom_to_internal_coordinates()
for residue in _integrated_residue_iterator(s):
    try:
        _ric=residue.internal_coord.get_angle
    except:
        continue
    print(residue.full_id,end='\t')
    i=1
    while _ric(f'chi{i}'):
        print(_ric(f'chi{i}'),end='\t')
        print(f'chi{i}')
        i+=1
    print('')

('1h2r.pdb', 0, 'S', (' ', 1, ' '))	39.56587291730228	chi1
64.2767556089065	chi2

('1h2r.pdb', 0, 'S', (' ', 2, ' '))	-149.68412609110598	chi1
-65.01568307604309	chi2
145.43922364439143	chi3

('1h2r.pdb', 0, 'S', (' ', 3, ' '))	
('1h2r.pdb', 0, 'S', (' ', 4, ' '))	30.661485743238526	chi1
-42.787999910006505	chi2

('1h2r.pdb', 0, 'S', (' ', 5, ' '))	39.374091210276674	chi1
-133.81518788559794	chi2
-54.61608793850407	chi3
90.79224263807927	chi4
-0.011706559159805658	chi5

('1h2r.pdb', 0, 'S', (' ', 6, ' '))	-64.51952199516526	chi1
140.8532565472902	chi2
60.680854155788985	chi3
-144.13277656680214	chi4
-0.5663747786576899	chi5

('1h2r.pdb', 0, 'S', (' ', 7, ' '))	-27.04452306235991	chi1
41.896509067015856	chi2

('1h2r.pdb', 0, 'S', (' ', 8, ' '))	178.409813244599	chi1

('1h2r.pdb', 0, 'S', (' ', 9, ' '))	-177.72023772245774	chi1

('1h2r.pdb', 0, 'S', (' ', 10, ' '))	168.83513760350615	chi1

('1h2r.pdb', 0, 'S', (' ', 11, ' '))	-161.89505105507507	chi1
113.5663624263273	chi2

('1h2r.pdb', 

In [23]:
_ric('chi4')